In [1]:
from NeuralProcessor import * 

In [2]:
## filepaths and names, change this to your file location. 
data_path = '/Users/seonghyunyoon/Developer/nptl/lower_frequencies/Data'
file_name_with_audio = 't12_t12.2022.08.13_Data__NSP1_NSP Data_1_neuralProcess_Complete_bld(001)002.ns5'
file_name_without_audio = 't12_t12.2022.08.13_Data__NSP2_NSP Data_1_neuralProcess_Complete_bld(001).ns5'

# The read_ns5_file function loads neural data from the ns5 file. 
# It can be used for both files with and without audio data. 
raw_neural_without_audio = read_ns5_file(f'{data_path}/{file_name_without_audio}', n_channels=128)
raw_neural, audio = read_ns5_file(f'{data_path}/{file_name_with_audio}', n_channels=128, include_audio=True)


t12_t12.2022.08.13_Data__NSP2_NSP Data_1_neuralProcess_Complete_bld(001).ns5 opened

t12_t12.2022.08.13_Data__NSP2_NSP Data_1_neuralProcess_Complete_bld(001).ns5 closed

t12_t12.2022.08.13_Data__NSP1_NSP Data_1_neuralProcess_Complete_bld(001)002.ns5 opened

t12_t12.2022.08.13_Data__NSP1_NSP Data_1_neuralProcess_Complete_bld(001)002.ns5 closed


In [76]:
FS=30000

In [88]:
raw_neural1 = raw_neural[600000:700000]
raw_neural2 = raw_neural[500000:600000]
#raw_neural2 = raw_neural_without_audio[600000:700000]

In [89]:
LRRfilter = ReReferenceFilter("lrr")
LRRfilter.set_weights_with(raw_neural1, 1, 128, FS)

In [90]:
Diagnostic_processor = NeuralProcessor({
    "processes": [
        ChevyshevFilter("lowpass", Wn=[int(FS * 0.2)], rp=0.05, ord=8, fs=FS, non_causal=True), 
        Downsampler(ds_factor=2),
        ButterworthFilter("bandpass", Wn=[250, 4900], ord=4, fs=FS, non_causal=True)
    ]
})

In [91]:
diagnostic_neural = Diagnostic_processor(raw_neural1, 1, 128, FS, verbose=True)['lfp']

Denoising steps
[1/3]: lowpass filtered (Chevyshev type I) for Wn=[6000]Hz
[2/3]: downsampled by factor of 2
[3/3]: bandpass filtered (Butterworth) for Wn=[250, 4900]Hz
Feature extraction steps
[1/1] Local field potentials extracted


In [92]:
threshold = ThresholdCrossingExtractor([],0,0,"rms").get_raw_threshold(diagnostic_neural)

In [93]:
threshold

array([20.244, 14.782, 13.591, 13.891, 23.238, 15.419, 25.579, 29.752,
       16.871, 17.178, 32.436, 17.995, 26.896, 36.182, 42.015, 26.135,
       48.356, 27.355, 15.762, 19.091, 26.639, 68.675, 31.811, 47.562,
       34.768, 27.396, 16.243, 18.388, 15.23 , 14.796, 18.073, 13.772,
       20.244, 27.659, 31.408, 20.512, 21.243, 18.572, 27.49 , 25.136,
       33.206, 36.674, 21.931, 34.944, 41.026, 28.113, 29.121, 54.572,
       50.377, 35.034, 21.071, 27.968, 35.156, 36.237, 42.841, 37.056,
       23.943, 21.681, 29.661, 26.604, 17.218, 27.043, 21.564, 30.965,
       21.086, 23.908, 21.629, 22.823, 23.471, 20.179, 32.199, 20.975,
       30.62 , 23.927, 24.757, 34.311, 28.213, 33.913, 26.465, 33.834,
       22.027, 18.991, 29.16 , 36.091, 30.32 , 31.324, 37.593, 46.405,
       20.206, 19.78 , 20.925, 24.403, 33.337, 41.046, 26.936, 22.275,
       28.774, 32.271, 30.395, 32.139, 21.502, 36.499, 36.887, 22.488,
       31.373, 24.42 , 23.815, 38.803, 29.327, 24.154, 33.387, 13.97 ,
      

In [94]:
Feature_extractor = NeuralProcessor({
    "processes": [
        LRRfilter,
        ChevyshevFilter("lowpass", Wn=[int(FS * 0.2)], rp=0.05, ord=8, fs=FS, non_causal=True), 
        Downsampler(ds_factor=2),
        ButterworthFilter("bandpass", Wn=[250, 4900], ord=4, fs=FS, non_causal=True)
    ], 
    "thresh_mults": [-3.5, -4.5], 
    "thresh_values": threshold, 
    "spike_pow_bands": ['full'], 
    "bin_size": 20, 
    "shift_size": 20
})

In [95]:
out = Feature_extractor(raw_neural2, 1, 128, FS, verbose=True)

Denoising steps
[1/4]: linear regression reference (LRR) filtered
[2/4]: lowpass filtered (Chevyshev type I) for Wn=[6000]Hz
[3/4]: downsampled by factor of 2
[4/4]: bandpass filtered (Butterworth) for Wn=[250, 4900]Hz
Feature extraction steps
[1/3] Local field potentials extracted
[2/3] Threshold crossing counts extracted for multipliers [-3.5, -4.5]
[3/3] Spiking bancpower extracted for frequency bands ['full']Hz


In [96]:
out['lfp']

array([[  3.913,   2.471,  -3.655, ...,   0.329,  -0.904,   1.26 ],
       [ -0.29 ,  47.42 ,  12.283, ...,  -1.07 ,  23.287, -29.432],
       [-22.928,  67.085,  11.862, ...,  -3.965,  24.653, -46.607],
       ...,
       [  5.601, -22.046,  23.649, ..., -15.674,   0.749,  11.694],
       [  6.161, -13.859,   7.912, ...,  -0.724,   9.766,  -1.258],
       [ -0.747,   3.306,  -6.709, ...,  -0.332,  -3.119,  -1.554]])

In [98]:
out['threshold_-3.5']

array([[ 8.,  0., 17., ...,  5.,  0.,  8.],
       [ 7.,  1., 10., ...,  2.,  0.,  7.],
       [ 3.,  0., 10., ...,  3.,  1.,  2.],
       ...,
       [10.,  0.,  2., ...,  2.,  0.,  0.],
       [14.,  1., 11., ..., 10.,  1.,  3.],
       [ 2.,  0.,  5., ...,  3.,  2.,  2.]], dtype=float32)

In [51]:
out

{'lfp': array([[  0.107,  -0.569,  -0.698, ...,  -1.338,  -4.073, -10.353],
        [ -9.437, -23.985,   3.003, ..., -31.082,  43.573,  33.721],
        [ -2.719, -36.178,   1.289, ..., -62.298,  11.153,  53.642],
        ...,
        [-39.88 , -26.555,  34.439, ..., -47.725, -36.824, -43.109],
        [ -7.897, -33.401,  21.142, ..., -26.313, -11.926, -16.702],
        [ -3.032,  -5.17 ,   0.5  , ...,   2.028,  -3.794,   0.531]]),
 'threshold_-3.5': array([[ 0., 18.,  9., ..., 87.,  2., 44.],
        [ 0.,  4., 20., ..., 65.,  8., 38.],
        [ 0.,  8., 12., ..., 72.,  7., 38.],
        ...,
        [ 0.,  4., 22., ..., 69.,  8., 35.],
        [ 0.,  6., 23., ..., 72.,  4., 35.],
        [ 2.,  7.,  9., ..., 49.,  6., 20.]], dtype=float32),
 'threshold_-4.5': array([[ 0.,  7.,  2., ..., 72.,  0., 21.],
        [ 0.,  3., 10., ..., 51.,  0., 21.],
        [ 0.,  0.,  6., ..., 60.,  4., 23.],
        ...,
        [ 0.,  3.,  9., ..., 46.,  4., 17.],
        [ 0.,  3., 13., ..., 58.,  